In [ ]:
import os
import glob

import pandas as pd
import matplotlib.pyplot as plt

# Local -> all other imports in source files
import settings.custom
from prophandler import PropHandler

from visualizations import prop_ratio
from backup_projections import load_backup_projections, create_historical_props
from private import _load_stokastic_data
from __utils import load_injuries, output_msgs

########################################################################################
# CONTEST SETTINGS
########################################################################################
SITE = "draftkings"
MODE = "classic"
# MODE = 'showdown'
########################################################################################
# PLAYER SETTINGS
########################################################################################
# Won't work unless you input a PATH (default is my local machine) to FanDuel contest file 
# SITE doesn't matter, DK does not have injury columns, disable if you want
DROP = load_injuries() + []
# Historical prop dataset
HISTORICAL = create_historical_props()
"""
- Each of EDITS (fpts) and OWNERSHIP (own) are possible inputs of type: dict[str,float]
- Players fpts will always be based on props if available (unless overriden, below)
- Players without props will fall back to the value passed for them in EDITS
- MUST use own .json files
"""
# JSON file of player projections to use if no props currently listed
# BACKUP_EDITS_FILE = '../data/backup_projections.json'
# Using load_backup_projections only reads files with just {name:fpts} so no ownership returned
# EDITS, OWNERSHIP = load_backup_projections(file=BACKUP_EDITS_FILE), dict()
# Premium third party data for personal use, hidden in private file
EDITS, OWNERSHIP = _load_stokastic_data(drop=DROP)
########################################################################################
# Prop projections always take precedence unless overriden
OVERRIDE_EDITS = {}
# If defined as a list, player's backup edits will override props
# OVERRIDE_EDITS = []
########################################################################################

In [ ]:
df = PropHandler(
    site=SITE,
    mode=MODE,
    edits=EDITS,
    ownership=OWNERSHIP,
    drop=DROP,
    override_edits=OVERRIDE_EDITS
).load(
    update=True,
    sort="e_fpts/$",
).reset_index().assign(
    e_ratio=lambda df_: (df_.e_fpts / df_.fpts).round(3),
    historical=lambda df_: df_.name.map(lambda name_: HISTORICAL.get(name_, 0.0)),
    hist_diff=lambda df_: df_.fpts-df_.historical,
    hist_ratio=lambda df_: (df_.fpts/df_.historical).round(2),
    rank=lambda df_: df_.index+1
).set_index('name')



In [ ]:
if not df.loc[df.props == '---'].empty:
    try:
        fig = prop_ratio(df, cutoff=0.0)
        display(fig)  # Only display once
        plt.close(fig)
    except ValueError:
        print('No props available. Using backup projections for all players.\n')
else:
    output_msgs('All players with props', char='=')

display(df)

In [ ]:
team_dfs = {
    team:
    df.loc[df['team'] == team]
    for team in df['team'].drop_duplicates()
}

In [ ]:
import numpy as np
# team_dfs[str(np.random.choice(list(team_dfs.keys())))]

#### `handler.constant_scrape()`
##### Repeatedly scrapes props, making sure always as up to date as possible and saving behind the scenes.
##### Caveats include the fact it has an infinite loop built in and that it is pretty useless when it comes to playoffs since books much more on top of lines then.

In [ ]:
# df = PropHandler(
#     SITE,
#     MODE,
#     edits=EDITS,
#     drop=DROP,
#     constant=True,
#     # override_edits=OVERRIDE_EDITS
# ).constant_scrape(verbose=0) 

In [ ]:
team_dfs['UTA']